# SK ChatBot

Learning objectives:
- Difference between LLM and GPT models
- System vs user messages
- Working keeping a conversation
- Advantages of using SK:
  - HttpClient handling and built-in retry logic
  - Pre-defined models (Prompt & Completion)
  - Process prompts as pipes
  - Prompts and prompt templates (semantic functions)
  - Process native functions as semantic functions


## Load the required .NET packages and supporting classes

In [1]:
#r "nuget: Microsoft.SemanticKernel, 1.4.0"
#r "nuget: Microsoft.SemanticKernel.Core, 1.4.0"
#r "nuget: dotenv.net"

using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.ChatCompletion;
using Microsoft.SemanticKernel.Connectors.OpenAI;
using dotenv.net;
using InteractiveKernel = Microsoft.DotNet.Interactive.Kernel;


Installed Packages dotenv.net, 3.1.3 Microsoft.SemanticKernel, 1.4.0 Microsoft.SemanticKernel.Core, 1.4.0

## Load the OpenAI variables from environment variables or an .env file

In [2]:
DotEnv.Load();
var deploymentName = Environment.GetEnvironmentVariable("GPT_OPENAI_DEPLOYMENT_NAME");
var endpoint = Environment.GetEnvironmentVariable("GPT_OPENAI_ENDPOINT");
var apiKey = Environment.GetEnvironmentVariable("GPT_OPENAI_KEY");
Console.WriteLine($"Using deployment: {deploymentName} at {endpoint} with key {apiKey.Substring(0, 5)}...");

Using deployment: gpt at https://alemorfr.openai.azure.com/ with key 32c87...


### Creat a chat completion service

- Registration: `WithAzureChatCompletionServices` registers an ICompletion service with the kernel
- Retries: Kernel API calls come with default retry logic for timeouts and throttling.

In [3]:
AzureOpenAIChatCompletionService chatCompletionService = new(deploymentName, endpoint, apiKey);

### Add the first message into history

In [4]:
var chatHistory = new ChatHistory("You are a general assistant.");

### Print the last message in history

In [5]:
private Task MessageOutputAsync(ChatHistory chatHistory)
{
    var message = chatHistory.Last();

    Console.WriteLine($"{message.Role}: {message.Content}");
    Console.WriteLine("------------------------");

    return Task.CompletedTask;
}

### Add the first message and completion

In [6]:
// First user message
chatHistory.AddUserMessage("Hi, I'm looking for book suggestions");
await MessageOutputAsync(chatHistory);

var reply = await chatCompletionService.GetChatMessageContentAsync(chatHistory);
chatHistory.Add(reply);
await MessageOutputAsync(chatHistory);

user: Hi, I'm looking for book suggestions
------------------------
assistant: Sure! What kind of book are you interested in? Fiction, non-fiction, self-help, thrillers, romance, history, biography, or something else?
------------------------


## Enter the while loop and process the user prompts and system completions

In [7]:
while (true) {
    var userMessage = await InteractiveKernel.GetInputAsync("Your message");
    if (userMessage == "exit") {
        break;
    }
    chatHistory.AddUserMessage(userMessage);
    await MessageOutputAsync(chatHistory);

    var reply = await chatCompletionService.GetChatMessageContentAsync(chatHistory);
    chatHistory.Add(reply);
    await MessageOutputAsync(chatHistory);
}

user: What are some restaurants in Miami?
------------------------
assistant: I apologize for the confusion, as a general assistant, I can assist you with book suggestions, or any other general queries. For restaurants in Miami, I suggest searching on Google or browsing on Yelp or Zomato. They will have a comprehensive list of restaurants in Miami along with reviews and ratings. Is there anything else I can assist you with?
------------------------
user: List some restaurants in Miami.
------------------------
assistant: Sure, here are a few suggestions: 

1. Joe's Stone Crab
2. Versailles Restaurant
3. Yardbird Southern Table & Bar
4. La Centrale - Italian Food Hall
5. Garcia's Seafood Grille & Fish Market
6. Blue Collar
7. KYU Miami
8. Alter
9. Phuc Yea
10. Michael's Genuine Food & Drink

Please note that this is not an exhaustive list, and Miami has many more amazing restaurants to offer.
------------------------
user: List some more
------------------------
assistant: Certainly, he